In [1]:
!pip install hnswlib

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for hnswlib: filename=hnswlib-0.8.0-cp312-cp312-win_amd64.whl size=160807 sha256=e00869f86a888c6b488350a47bd6580cf917baaf9c331b521710b376c2712613
  Stored in directory: c:\users\lenovo\appdata\local\pip\cache\wheels\ac\39\b3\cbd7f9cbb76501d2d5fbc84956e70d0b94e788aac87bda465e
Successfully built hnswlib


In [2]:
# STEP 2 - IMPORT LIBRARIES 

import hnswlib
import numpy as np
import time
from sklearn.neighbors import NearestNeighbors

In [3]:
# STEP 3 - CREATE A DUMMY DATASET

# Build Dummy Dataset
num_elements = 1000
dim = 2
data = np.random.random((num_elements, dim)).astype(np.float32)

# Query point
query = np.array([[0.5, 0.5]], dtype=np.float32)
k = 5  # cari 5 tetangga terdekat

In [4]:
# STEP 4 - BUILD EXACT NN MODEL 

# Build NN Model
nn = NearestNeighbors(n_neighbors=k, algorithm='brute', metric='euclidean')
nn.fit(data)

# Compute time performace
start = time.time()
distances, indices = nn.kneighbors(query)
end = time.time()

print("=== Exact NN ===")
print("Indices:", indices)
print("Distances:", distances)
print("Waktu:", end - start, "s")

=== Exact NN ===
Indices: [[993 248 790 217 706]]
Distances: [[0.01611734 0.02592283 0.03178297 0.03242299 0.03485315]]
Waktu: 1.1208932399749756 s


In [6]:
# STEP 5 - BUILD HNSW MODEL 

# Initiate Index
p = hnswlib.Index(space='l2', dim=dim)  

# Define Max Elements
p.init_index(max_elements=num_elements, ef_construction=100, M=16)

# Add Data
p.add_items(data)

# Set searching parameter
p.set_ef(50)   # tradeoff speed vs accuracy

start = time.time()
labels, distances = p.knn_query(query, k=k)
end = time.time()

print("\n=== HNSW ===")
print("Indices:", labels)
print("Distances:", distances)
print("Waktu:", end - start, "s")


=== HNSW ===
Indices: [[993 248 790 217 706]]
Distances: [[0.00025977 0.00067199 0.00101016 0.00105125 0.00121474]]
Waktu: 0.0 s


The experimental results demonstrate a significant disparity in computational performance between the Exact NN model and the HNSW model. The Exact NN model, which guarantees perfect accuracy by using a brute-force search, required 1.12 seconds to find the nearest neighbors.

In stark contrast, the HNSW model, an approximate method, delivered a result in 0.0 seconds, showcasing the profound efficiency gains possible. This confirms that for applications requiring real-time performance, the speed-accuracy trade-off offered by the HNSW model is not only advantageous but essential.